In [35]:
import pyodbc 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [36]:
server = 'BHRAMENDRA\SQLEXPRESS01'
database = 'test_Squadcast'
conn = pyodbc.connect('Driver={SQL Server};SERVER='+server+';DATABASE='+database+';Treusted_connection=yes;')
conn.autocommit=True
cursor = conn.cursor()

In [37]:
df_movie = pd.read_sql("select * from [dbo].[movie]",conn)
df_rating = pd.read_sql("select * from [dbo].[rating]",conn)


In [38]:
df_movie.head(10)

,id,title,year,country,genre,director,minutes,poster
0,3112654,Mea culpa,2014,France,"Action, Thriller",Fred Cavayé,90,http://ia.media-imdb.com/images/M/MV5BNTQ0ODAw...
1,1571222,A Dangerous Method,2011,"UK, Germany, Canada, Switzerland","Biography, Drama, Thriller",David Cronenberg,99,http://ia.media-imdb.com/images/M/MV5BMTU5Mjk3...
2,2756032,The One I Love,2014,USA,"Comedy, Drama, Romance",Charlie McDowell,91,http://ia.media-imdb.com/images/M/MV5BMTgzODMy...
3,2370248,Short Term 12,2013,USA,Drama,Destin Daniel Cretton,96,http://ia.media-imdb.com/images/M/MV5BMTEwNjE2...
4,57171,"Yesterday, Today and Tomorrow",1963,"Italy, France","Comedy, Romance",Vittorio De Sica,119,http://ia.media-imdb.com/images/M/MV5BMTY0NDc4...
5,1433540,A Town Called Panic,2009,"Belgium, Luxembourg, France","Animation, Adventure, Comedy","Stéphane Aubier, Vincent Patar",75,http://ia.media-imdb.com/images/M/MV5BMjEyODIy...
6,1433811,Disconnect,2012,USA,"Drama, Thriller",Henry Alex Rubin,115,http://ia.media-imdb.com/images/M/MV5BMjA2MTEz...
7,1608368,House Hunting,2013,USA,"Horror, Mystery, Thriller",Eric Hurt,102,http://ia.media-imdb.com/images/M/MV5BNTI2NDcx...
8,1220634,Resident Evil: Afterlife,2010,"Germany, France, USA, Canada","Action, Adventure, Horror",Paul W.S. Anderson,97,http://ia.media-imdb.com/images/M/MV5BMTkxNzM3...
9,858479,Smart People,2008,USA,"Comedy, Drama, Romance",Noam Murro,95,http://ia.media-imdb.com/images/M/MV5BMTg1NTI3...


In [39]:
df_rating.head(10)

,rater_id,movie_id,rating,time
0,1,68646,10,1381620027
1,1,113277,10,1379466669
2,2,422720,8,1412178746
3,2,454876,8,1394818630
4,2,790636,7,1389963947
5,2,816711,8,1379963769
6,2,1091191,7,1391173869
7,2,1103275,7,1408192129
8,2,1322269,7,1391529691
9,2,1433811,8,1380453043


In [41]:
# Merge the two DataFrames on the 'movie_id' column
merged_df = pd.merge(df_movie, df_rating, left_on='id', right_on='movie_id')

In [40]:
# Que a : Top 5 Movie Titles: Sort and print the top 5 movie titles based on the following criteria:
# ● Duration
# ● Year of Release
# ● Average rating (consider movies with minimum 5 ratings)
# ● Number of ratings given

In [42]:


# Calculate average rating and number of ratings for each movie
movie_stats = merged_df.groupby('title').agg({'minutes': 'first', 'year': 'first', 'rating': ['mean', 'count']})

# Filter movies with at least 5 ratings
movie_stats = movie_stats[movie_stats['rating']['count'] >= 5]

# Sort by duration, year, average rating, and number of ratings
sorted_movies = movie_stats.sort_values(by=[('minutes', 'first'), ('year', 'first'), ('rating', 'mean'), ('rating', 'count')], ascending=[False, True, False, False])

# Print the top 5 movie titles
top_5_movies = sorted_movies.head(5)
print(top_5_movies)


                                              minutes  year    rating      
                                                first first      mean count
title                                                                      
The Godfather: Part II                            202  1974  9.800000     5
The Lord of the Rings: The Return of the King     201  2003  9.600000     5
Schindler's List                                  195  1993  8.333333     6
The Green Mile                                    189  1999  9.000000     9
The Wolf of Wall Street                           180  2013  8.376812    69


In [43]:
# Que b : Number of Unique Raters: Determine and print the count of unique rater IDs.

In [44]:
unique_raters_count = df_rating['rater_id'].nunique()

print(f"Number of Unique Raters: {unique_raters_count}")

Number of Unique Raters: 1048


In [45]:
# Que c: Top 5 Rater IDs: Sort and print the top 5 rater IDs based on:
#     ● Most movies rated
#     ● Highest Average rating given (consider raters with min 5 ratings)

In [46]:
# Calculate the number of ratings and average rating for each rater
rater_stats = df_rating.groupby('rater_id').agg({'movie_id': 'count', 'rating': ['mean', 'count']})

# Filter raters with at least 5 ratings
rater_stats = rater_stats[rater_stats['rating']['count'] >= 5]

# Sort by most movies rated and highest average rating
sorted_raters = rater_stats.sort_values(by=[('movie_id', 'count'), ('rating', 'mean')], ascending=[False, False])

# Print the top 5 rater IDs
top_5_raters = sorted_raters.head(5)
print(top_5_raters)


         movie_id    rating      
            count      mean count
rater_id                         
735           314  6.907643   314
769           209  7.220096   209
512           188  7.484043   188
800           186  7.569892   186
82            159  6.459119   159


In [47]:
# Que d : Top Rated Movie:
# - Find and print the top-rated movies by:
# ● Director 'Michael Bay',
# ● 'Comedy',
# ● In the year 2013
# ● In India (consider movies with a minimum of 5 ratings).

In [48]:

# Filter movies directed by 'Michael Bay', 'Comedy' genre, in the year 2013, and in India
filtered_movies = merged_df[
    (merged_df['director'] == 'Michael Bay') &
    (merged_df['genre'].str.contains('Comedy', case=False)) &
    (merged_df['year'] == 2013) &
    (merged_df['country'] == 'India') &
    (merged_df['rating'] >= 5)  # Consider movies with at least 5 ratings
]

# Group by movie title and calculate average rating
top_rated_movies = filtered_movies.groupby('title')['rating'].mean()

# Sort in descending order to get the top-rated movies
top_rated_movies = top_rated_movies.sort_values(ascending=False)

# Print the top-rated movies
print(top_rated_movies.head())


Series([], Name: rating, dtype: float64)


In [56]:
# Que e : Favorite Movie Genre of Rater ID 1040: Determine and print the favorite movie genre
# for the rater with ID 1040 (defined as the genre of the movie the rater has rated most often).


In [49]:
# Filter ratings by rater ID 1040
ratings_by_rater_1040 = merged_df[merged_df['rater_id'] == 1040]

# Count the occurrences of each genre
genre_counts = ratings_by_rater_1040['genre'].value_counts()

# Determine the favorite genre (genre with the highest count)
favorite_genre = genre_counts.idxmax()

# Print the favorite genre
print(f"Favorite Movie Genre of Rater ID 1040: {favorite_genre}")


Favorite Movie Genre of Rater ID 1040: Action, Adventure, Sci-Fi


In [54]:
# Que f :  Highest Average Rating for a Movie Genre by Rater ID 1040: Find and print the
# highest average rating for a movie genre given by the rater with ID 1040 (consider genres with a
# minimum of 5 ratings).


In [50]:
# Filter ratings by rater ID 1040
ratings_by_rater_1040 = merged_df[merged_df['rater_id'] == 1040]

# Group by genre and calculate average rating
genre_avg_ratings = ratings_by_rater_1040.groupby('genre')['rating'].agg(['mean', 'count'])

# Filter genres with at least 5 ratings
genre_avg_ratings = genre_avg_ratings[genre_avg_ratings['count'] >= 5]

# Find the genre with the highest average rating
highest_rated_genre = genre_avg_ratings['mean'].idxmax()

# Print the highest average rating and corresponding genre
print(f"Highest Average Rating for a Movie Genre by Rater ID 1040: {highest_rated_genre} with rating {genre_avg_ratings.loc[highest_rated_genre, 'mean']}")


Highest Average Rating for a Movie Genre by Rater ID 1040: Action, Adventure, Sci-Fi with rating 8.454545454545455


In [55]:
# Que g :  Year with Second-Highest Number of Action Movies: Identify and print the year with
# the second-highest number of action movies from the USA that received an average rating of
# 6.5 or higher and had a runtime of less than 120 minutes.

In [51]:
# Filter action movies from the USA with average rating >= 6.5 and runtime < 120 minutes
filtered_movies = merged_df[
    (merged_df['genre'].str.contains('Action', case=False)) &
    (merged_df['country'] == 'USA') &
    (merged_df['rating'] >= 6.5) &
    (merged_df['minutes'] < 120)
]

# Group by year and count the number of movies
yearly_counts = filtered_movies.groupby('year').size()

# Identify the year with the second-highest number of action movies
second_highest_year = yearly_counts.nlargest(2).idxmin()

# Print the result
print(f"Year with Second-Highest Number of Action Movies: {second_highest_year}")


Year with Second-Highest Number of Action Movies: 2014


In [52]:
# Que h : Count of Movies with High Ratings: Count and print the number of movies that have
# received at least five reviews with a rating of 7 or higher.


In [53]:
# Filter movies with at least five reviews and a rating of 7 or higher
high_rated_movies = df_rating[df_rating['rating'] >= 7].groupby('movie_id').size() >= 5

# Count the number of movies meeting the criteria
num_high_rated_movies = high_rated_movies.sum()

# Print the result
print(f"Count of Movies with High Ratings (at least five reviews with a rating of 7 or higher): {num_high_rated_movies}")


Count of Movies with High Ratings (at least five reviews with a rating of 7 or higher): 338
